In [1]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kojikojipro","key":"04c6e3329de941099c00e7a9a89a384a"}'}

In [0]:
# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
# kaggle.jsonの権限変更
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# Google Colabでデータセットをダウンロード
!kaggle competitions download -c digit-recognizer

# データセットを解凍
!unzip train.csv.zip
!unzip test.csv.zip

 82% 5.00M/6.09M [00:00<00:00, 20.7MB/s]
100% 6.09M/6.09M [00:00<00:00, 24.2MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 78.5MB/s]
 55% 5.00M/9.16M [00:00<00:00, 25.1MB/s]
100% 9.16M/9.16M [00:00<00:00, 36.3MB/s]
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
# Usual imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf

# Network building and training
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, SeparableConv2D, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

In [0]:
train = pd.read_csv('train.csv').astype(float)
test_x_raw = pd.read_csv('test.csv').astype(float).to_numpy()

train_y_raw = train.loc[:, 'label'].astype(int).to_numpy()
train_x_raw = train.drop('label', axis=1).to_numpy()

train_x_raw = np.reshape(train_x_raw, (train_x_raw.shape[0], 28, 28, 1))
test_x_raw = np.reshape(test_x_raw, (test_x_raw.shape[0], 28, 28, 1))

In [6]:
repeat = 2
train_x = train_x_raw.repeat(repeat, axis=1).repeat(repeat, axis=2)
test_x = test_x_raw.repeat(repeat, axis=1).repeat(repeat, axis=2)

train_x = train_x / 255.0
test_x = test_x / 255.0

print("Training set size:", train_x.shape)
print("Testing set size:", test_x.shape)

Training set size: (42000, 56, 56, 1)
Testing set size: (28000, 56, 56, 1)


In [0]:
train_y_one_hot = to_categorical(train_y_raw)

In [8]:
input_shape = train_x[0].shape
img_input = Input(shape = input_shape, name='input')

conv1_filters = 32
conv1_input = img_input
conv1_1 = SeparableConv2D(conv1_filters, kernel_size=(3, 5), padding='same', activation='relu', name='conv1_1')(conv1_input)
norm1_1 = BatchNormalization(name='norm1_1')(conv1_1)
conv1_2 = SeparableConv2D(conv1_filters, kernel_size=(5, 3), padding='same', activation='relu', name='conv1_2')(conv1_input)
norm1_2 = BatchNormalization(name='norm1_2')(conv1_2)
conv1_3 = SeparableConv2D(conv1_filters, kernel_size=(5, 7), padding='same', activation='relu', name='conv1_3')(conv1_input)
norm1_3 = BatchNormalization(name='norm1_3')(conv1_3)
conv1_4 = SeparableConv2D(conv1_filters, kernel_size=(7, 5), padding='same', activation='relu', name='conv1_4')(conv1_input)
norm1_4 = BatchNormalization(name='norm1_4')(conv1_4)

multiplied = Multiply()([norm1_1, norm1_2, norm1_3, norm1_4])
pool1 = MaxPooling2D(name='pool1')(multiplied)
dp1 = Dropout(0.2, name='dp1')(pool1)

conv2 = SeparableConv2D(64, (3, 3), padding='same', activation='relu',name='conv2')(dp1)
conv2 = SeparableConv2D(64, (3, 3), padding='same', activation='relu',name='conv2')(dp1)
norm2 = BatchNormalization(name='norm2')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), name='pool2')(norm2)
dp2 = Dropout(0.4, name='dp2')(pool2)
conv3 = SeparableConv2D(128, (3, 3), activation='relu', name='conv3')(dp2)
norm3 = BatchNormalization(name='norm3')(conv3)
pool3_1 = MaxPooling2D(pool_size=(2, 2), name='pool3_1')(norm3)
pool3_2 = MaxPooling2D(pool_size=(2, 2), name='pool3_2')(pool3_1)
dp3 = Dropout(0.4, name='dp3')(pool3_2)

fl = Flatten()(dp3)

fc4 = Dense(128, activation='relu', name="fc4")(fl)
norm4 = BatchNormalization(name='norm4')(fc4)
dp4 = Dropout(0.5, name='dp4')(norm4)
fc5 = Dense(128, activation='relu', name="fc5")(dp4)
norm5 = BatchNormalization(name='norm5')(fc5)
dp5 = Dropout(0.5, name='dp5')(norm5)
output = Dense(10, activation='softmax', name="output")(dp5)

network = Model(img_input, output, name='CNN_classification')
network.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.005), metrics=['acc'])

network.summary()

Model: "CNN_classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 56, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv1_1 (SeparableConv2D)       (None, 56, 56, 32)   79          input[0][0]                      
__________________________________________________________________________________________________
conv1_2 (SeparableConv2D)       (None, 56, 56, 32)   79          input[0][0]                      
__________________________________________________________________________________________________
conv1_3 (SeparableConv2D)       (None, 56, 56, 32)   99          input[0][0]                      
_________________________________________________________________________________

In [0]:
# Google Driveをマウント
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

best_network = 'best_network.hdf5'

def get_id(name):
    file_lst = drive.ListFile({'q': "'root' in parents and trashed=False"}).GetList()
    for f in file_lst:
        if f['title'] == name:
            return f['id']

def create_file(name):
    _id = get_id(name)
    _file = drive.CreateFile({'id': _id})
    return _file

def download(name):
    _file = create_file(name)
    _file.GetContentFile(name)
    return _file

In [0]:
# 初回のみGoogle Driveにファイル作成
'''
nw_file = drive.CreateFile({'title': best_network})
nw_file.Upload()
'''
# Google Driveからダウンロード
nw_file = download(best_network)
network.load_weights(best_network)

In [0]:
# 学習
epochs = 1000

cb = [
    ModelCheckpoint(
        "best_network.hdf5", monitor='val_acc', verbose=2, 
        save_best_only=True, save_weights_only=False, mode='auto'),
]

H = network.fit(
    train_x, train_y_one_hot,
    batch_size=1000, epochs=epochs,
    validation_split=0.4 , shuffle=True,
    callbacks=cb,
    verbose=2)

Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.99429, saving model to best_network.hdf5
26/26 - 6s - loss: 0.0091 - acc: 0.9971 - val_loss: 0.0324 - val_acc: 0.9943
Epoch 2/1000

Epoch 00002: val_acc improved from 0.99429 to 0.99446, saving model to best_network.hdf5
26/26 - 5s - loss: 0.0079 - acc: 0.9973 - val_loss: 0.0276 - val_acc: 0.9945
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.99446
26/26 - 5s - loss: 0.0086 - acc: 0.9976 - val_loss: 0.0285 - val_acc: 0.9944
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.99446
26/26 - 5s - loss: 0.0106 - acc: 0.9966 - val_loss: 0.0291 - val_acc: 0.9937
Epoch 5/1000


In [0]:
# GoogleDriveに保存
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nw_file = create_file(best_network)
nw_file.SetContentFile(best_network)
nw_file.Upload()

In [0]:
plt.plot(H.history['loss'], label="loss")
plt.plot(H.history['val_loss'], label="val_loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("loss vs epochs")
plt.legend()
plt.show()

plt.plot(H.history['acc'], label="acc")
plt.plot(H.history['val_acc'], label="val_acc")
plt.xlabel("epochs")
plt.ylabel("acc")
plt.title("Accuracy vs epochs")
plt.legend()
plt.show()

In [0]:
network.load_weights(best_network)

y_pred_one_hot = network.predict(test_x)
y_pred = np.argmax(y_pred_one_hot, axis=1)

submission = pd.DataFrame()
submission['ImageId'] = [i for i in range(1, len(y_pred) + 1)]
submission['Label'] = y_pred
submission.to_csv('pred.csv', index=False)

In [0]:
!kaggle competitions submit -c digit-recognizer -f pred.csv